<a href="https://colab.research.google.com/github/Yin-Tzu/predict_DA_reaction_product/blob/main/%E4%B8%BB%E8%A6%81%E5%9F%B7%E8%A1%8C%E7%A8%8B%E5%BC%8F/(%E6%AD%A5%E9%A9%9F%E5%85%AD%E5%BE%8C%E5%8D%8A%E9%83%A8%E5%88%86%EF%BC%8C%E9%80%A3%E6%8E%A5%E5%8F%96%E4%BB%A3%E5%9F%BA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from PIL import Image
import copy
import random
from rdkit.Chem import AllChem, Draw
from rdkit import Chem
import math
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolDrawing, DrawingOptions

import py3Dmol
from IPython.display import display

def differentmolecule(ring,sub1,sub2,sub3,sub4,Hring,Lring):
  #找出支鏈鍵結位置
  def abranchposition(C,molecule):
    C2connectatom=[]
    #print(C,'C2')
    if len(C)>5:
      for i in C:
        #print([ i for i in [x.GetAtomicNum() for x in combo.GetAtomWithIdx(i).GetNeighbors()] if i not in [6] ], combo.GetAtomWithIdx(i).GetSymbol())
        if (molecule.GetAtomWithIdx(i).GetSymbol()!='H') and len([ i for i in [x.GetAtomicNum() for x in molecule.GetAtomWithIdx(i).GetNeighbors()] if i not in [6] ])==2 and len([ i for i in [x.GetAtomicNum() for x in molecule.GetAtomWithIdx(i).GetNeighbors()] if i not in [1,1] ])==1:
          C2connectatom.append(i)  # 第二個分子的雙鍵的邊邊
    else:
      for i in C:
        if (molecule.GetAtomWithIdx(i).GetSymbol()!='H'):
          C2connectatom=copy.deepcopy([i])
    return C2connectatom[0]

  def ringposition(molecule, HOMOring, LUMOring):
    PD,allconnectidx = [],[]  # 找環
    for Bond in molecule.GetBonds():  # 環
      if (int(Bond.GetBondTypeAsDouble())==2) and (Bond.IsInRing()==True):  # 若是雙鍵而且在環內 https://zhuanlan.zhihu.com/p/143111689主要的才有環
        PD.append(Bond.GetBeginAtomIdx())  # 加入鍵的起始原子id
        PD.append(Bond.GetEndAtomIdx())  # 加入鍵的後原子id
    #找出環中鍵結的所有位置
    ifHOMOC=HOMOring
    natomn = ifHOMOC//2
    if ifHOMOC%2==1:
      natomn+=1
    PP=copy.deepcopy(PD)
    aatomNeighborsCIdx=[]
    #print((natomn-1),PP[-1],'PP')
    natomncal = 0
    for i in range((natomn-1+3+(LUMOring-2))):
      aatomNeighborsAtomicNum=[x.GetAtomicNum()for x in molecule.GetAtomWithIdx(PP[-1]).GetNeighbors()]
      aatomNeighborsIdx=[x.GetIdx()for x in molecule.GetAtomWithIdx(PP[-1]).GetNeighbors()]
      for ii in range(len(aatomNeighborsAtomicNum)):
        if aatomNeighborsAtomicNum[ii] != 1:
          aatomNeighborsCIdx.append(aatomNeighborsIdx[ii])
          aatomNeighbor = [ i for i in aatomNeighborsCIdx if i not in PP ]  #差集
          
          if aatomNeighbor!=[]:
            if ((HOMOring+LUMOring)==4) and (i==0):
              allconnectidx.append(PP[-1])
            PP.append(aatomNeighbor[0])
            natomncal+=1
            if (natomn-1)== natomncal:
              allconnectidx.append(aatomNeighbor[0])
            elif ((natomn-1+1)==natomncal)or ((natomn-1+1+(LUMOring-2)+1)==natomncal)or((natomn-1+1+(LUMOring-2)+2)==natomncal):
              allconnectidx.append(aatomNeighbor[0])
            else:
              None
        if (((HOMOring+LUMOring)==4) or ((HOMOring+LUMOring)==5)):
          if i==(natomn-1+3+(LUMOring-2)-1) and (ii==(len(aatomNeighborsAtomicNum)-1)):
            allconnectidx.append(PP[0])
    #print(PP,aatomNeighbor,allconnectidx)
    return allconnectidx

  m = Chem.MolFromSmiles(ring)  # 要連接的分子
  H1 = Chem.MolFromSmiles(sub1)  # 要連接的分子
  L1 = Chem.MolFromSmiles(sub2)
  L2 = Chem.MolFromSmiles(sub3)
  H2 = Chem.MolFromSmiles(sub4)
  m=Chem.AddHs(m)
  H1,L1,L2,H2=Chem.AddHs(H1),Chem.AddHs(L1),Chem.AddHs(L2),Chem.AddHs(H2)

  #把支鏈氫去掉
  air=[]
  for j in [H1,L1,L2,H2]:
    C=[]
    for i in j.GetAtoms():  # C3是combo裡的所有原子編號
      C.append(i.GetIdx())
    #print(len(C))
    if len(C)>0:
      RH = abranchposition(C,j)
      edcombo = Chem.EditableMol(j)
      for i in [RH]:
        ind = [x.GetAtomicNum() for x in j.GetAtomWithIdx(i).GetNeighbors()].index(1)
        removeindex = [x.GetIdx() for x in j.GetAtomWithIdx(i).GetNeighbors()][ind]
      edcombo.RemoveAtom(removeindex)
      if j==H1:
        H1 = edcombo.GetMol()
      elif j==L1:
        L1 = edcombo.GetMol()
      elif j==L2:
        L2 = edcombo.GetMol()
      else:
        H2 = edcombo.GetMol()
    else:
      air.append([H1,L1,L2,H2].index(j))
  #print(air,'j')
  #把環的氫去掉
  for j in range(4):
    C=[]
    if j not in [k for k in air]:
      for i in m.GetAtoms():  # C3是combo裡的所有原子編號
        C.append(i.GetIdx())
      RH = ringposition(m,Hring,Lring)[j]
      edcombo = Chem.EditableMol(m)
      for i in [RH]:
        ind = [x.GetAtomicNum() for x in m.GetAtomWithIdx(i).GetNeighbors()].index(1)
        removeindex = [x.GetIdx() for x in m.GetAtomWithIdx(i).GetNeighbors()][ind]
      edcombo.RemoveAtom(removeindex)
      m = edcombo.GetMol()

  #放在同個屏幕
  combo = Chem.CombineMols(m,H1)  # 放在同一個屏幕李
  combo = Chem.CombineMols(combo,L1)
  combo = Chem.CombineMols(combo,L2)
  combo = Chem.CombineMols(combo,H2)

  idid,C1,C2=[],[],[]
  for i in combo.GetAtoms():  # C3是combo裡的所有原子編號
    idid.append(i.GetIdx())
  #print(idid)

  #把不同的分子區分開
  allC=[]
  tt=copy.deepcopy(idid)
  for iio in range(4-len(air)):
    for ii in range(6):  # 任何數字都可，是為了讓C1可以完整
      for i in range(len(tt)):  # # 分成兩個分子
        atom = combo.GetAtomWithIdx(tt[i])
        X = [x.GetIdx() for x in atom.GetNeighbors()]  # X 是所有鄰居編號
        #print(i,C1)
        if i==0 and C1==[]:
          #print(iio)
          C1 = copy.deepcopy(X)
        else:
          #if len(X)==2:
          for iji in range(len(X)):
            if (X[iji] in C1) or (tt[i] in C1):
              add = [ ioi for ioi in X if ioi not in C1 ]  #差集
              #print(add,'65')
              for ioi in add:
                C1.append(ioi)
              break
      #print(C1)
    if iio==0:
      tt = [ i for i in idid if i not in C1 ]  #差集
      C2=copy.deepcopy(C1)
      #print(tt,iio)
      if len(air)!=3:
        C1=copy.deepcopy([])
      else:
        C1=copy.deepcopy(tt)
        allC.append(C1)
      allC.append(C2)
    elif iio==1:
      ttt=copy.deepcopy(tt)
      tt = copy.deepcopy([ i for i in ttt if i not in C1 ])  #差集
      C3=copy.deepcopy(C1)
      #print(tt,iio)
      if len(air)!=2:
        C1=copy.deepcopy([])
      else:
        C1=copy.deepcopy(tt)
        allC.append(C1)
      allC.append(C3)

    elif iio==2:
      ttt=copy.deepcopy(tt)
      tt = copy.deepcopy([ i for i in ttt if i not in C1 ])  #差集
      #print(tt,iio)
      C4=copy.deepcopy(C1)
      if len(air)!=1:
        C1=copy.deepcopy([])
      else:
        C1=copy.deepcopy(tt)
        allC.append(C1)
        #print(C1,'C11')
      allC.append(C4)
    else:
      C5 = [ i for i in tt if i not in C1 ]  #差集
      allC.append(C5)
      allC.append(C1)

  #matches = C1[:-1].GetSubstructMatches(combo)
  #print(C1,'c1',C2,'C2',C3,'C3',C4,'C4')#,C5,'C5'

  #找到每個分子的大小索引，並把allC重新排列
  ringbranchs = []
  #ringbranchsnum=[]
  findCidx=0
  for j in range(4):
    for i in allC:
      #print(len(i))
      if len(i)>0:
        #print(i,'i')
        #print(min(i),findCidx,852)
        if min(i)==findCidx:
          #print(i,'i')
          findCidx = max(i)+1
          ringbranchs.append(i)
          #ringbranchsnum.append(1)
      else:
        #print(i,'i')
        """
        if len(ringbranchs)==5:
          break
        ringbranchs.append(i)
        Bidx=ringbranchs.index(i)
        """
        None
  #print(ringbranchs,'5')#,Bidx

  #環鍵結位置
  connectringposition = ringposition(combo,Hring,Lring)  # 不管如何出來都要有四個值
  #print(connectringposition,8)
  #print(air[::-1])
  #print(connectringposition)
  for a in air[::-1]:
    connectringposition.remove(connectringposition[a])
  #print(connectringposition)

  #支鏈位置
  BP,t=[],[]
  for atom in combo.GetAtoms():
    if atom.GetIdx() not in ringbranchs[0]:
      if (str(atom.GetHybridization())=='SP2') and (atom.GetTotalDegree()==2):#
        #print(atom.GetTotalDegree(),'321')
        BP.append(atom.GetIdx())
      else:
        for k in range(4-len(air)):
          for i in [ringbranchs[k]]:
            if len(i)==4:
              #print(i)
              if (atom.GetAtomicNum()==6)and (atom.GetIdx()in i):
                BP.append(atom.GetIdx())

  #為了讓鍵結的id可以隊的上
  #print(BP)
  TBP=[]
  for i in range(len(BP)):
    for j in range(len(BP)):
      if BP[j] in ringbranchs[(i+1)]:
        TBP.append(BP[j])
      else:
        None
  #print(TBP)
  edcombo = Chem.EditableMol(combo)
  for i in range(len(TBP)):
    edcombo.AddBond(TBP[i],connectringposition[i],order=Chem.rdchem.BondType.SINGLE)
  back = edcombo.GetMol()

  AllChem.EmbedMultipleConfs(back) #, numConfs=1, params=AllChem.ETKDG())
  Chem.AssignAtomChiralTagsFromStructure(back)
  for atom in back.GetAtoms():
        if atom.GetChiralTag() != Chem.rdchem.ChiralType.CHI_UNSPECIFIED:
            print(atom.GetAtomicNum(), atom.GetChiralTag())
  back=Chem.MolToMolBlock(back)

  p = py3Dmol.view(width=500,height=500)
  p.addModel(back,'sdf')
  p.setStyle({'sphere':{'scale':.3},'stick':{}}) # cartoon  stick  sphere  {'sphere':{'scale':.3},'stick':{}}
  p.zoomTo()
  display(p.show())
  #Draw.MolsToGridImage([H1,L1,L2,H2,m,combo,back], subImgSize=(200, 150)) 